# Before Getting Started 
- Make sure there is a file named either out.txt or alert_description.txt
- Otherwise
    - run python collect_data.py
    - run python get_alert_report.py or python get_alert_description.py on the terminal

# Opening the training data file

In [17]:
filename = "alert_description.txt" # Change this filename if necessary
file = open(filename) 
alert_msg = file.read()
file.close()

len(alert_msg)

3995915

# Prepare the mapping for the char rnn model
we are going to make a mapping of the character to certain number

In [8]:
# import the library that are needed
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
# The unique characters in the file
vocab = sorted(set(alert_msg))
print ('{} unique characters'.format(len(vocab)))

85 unique characters


In [41]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in alert_msg])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [42]:
print('{')
for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  \n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  '6' :  20,
  '7' :  21,
  '8' :  22,
  '9' :  23,
  ':' :  24,
  ';' :  25,
  '<' :  26,
  '=' :  27,
  '?' :  28,
  '@' :  29,
  'A' :  30,
  'B' :  31,
  'C' :  32,
  'D' :  33,
  'E' :  34,
  'F' :  35,
  'G' :  36,
  'H' :  37,
  'I' :  38,
  'J' :  39,
  'K' :  40,
  'L' :  41,
  'M' :  42,
  'N' :  43,
  'O' :  44,
  'P' :  45,
  'Q' :  46,
  'R' :  47,
  'S' :  48,
  'T' :  49,
  'U' :  50,
  'V' :  51,
  'W' :  52,
  'X' :  53,
  'Y' :  54,
  'Z' :  55,
  '[' :  56,
  ']' :  57,
  '_' :  58,
  'a' :  59,
  'b' :  60,
  'c' :  61,
  'd' :  62,
  'e' :  63,
  'f' :  64,
  'g' :  65,
  'h' :  66,
  'i' :  67,
  'j' :  68,
  'k' :  69,
  'l' :  70,
  'm' :  71,
  'n' :  72,
  'o' :  73,
  'p' :  74,
  'q' :  75,
  'r' :  7

In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(alert_msg[122:129]), text_as_int[122:129]))

'er At B' ---- characters mapped to int ---- > [63 76  1 30 78  1 31]


In [43]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(alert_msg)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [44]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [45]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [46]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Going to build the model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 1256

# Number of RNN units
rnn_units = 1024 # 512

In [29]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [30]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [37]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

NameError: name 'BATCH_SIZE' is not defined

In [38]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

NameError: name 'dataset' is not defined

In [32]:
model.summary()

NameError: name 'model' is not defined

In [28]:
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) 
#sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
#sampled_indices

In [30]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4425693


In [31]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS= 30

In [34]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/30
624/624 [==============================] - 157s 252ms/step - loss: 1.1285
Epoch 2/30
624/624 [==============================] - 151s 242ms/step - loss: 0.4672
Epoch 3/30
624/624 [==============================] - 153s 246ms/step - loss: 0.3930
Epoch 4/30
624/624 [==============================] - 158s 253ms/step - loss: 0.3599
Epoch 5/30
624/624 [==============================] - 155s 249ms/step - loss: 0.3434
Epoch 6/30
624/624 [==============================] - 159s 255ms/step - loss: 0.3337
Epoch 7/30
624/624 [==============================] - 155s 249ms/step - loss: 0.3302
Epoch 8/30
624/624 [==============================] - 157s 252ms/step - loss: 0.3284
Epoch 9/30
624/624 [==============================] - 165s 264ms/step - loss: 0.3292
Epoch 10/30
624/624 [==============================] - 152s 244ms/step - loss: 0.3295
Epoch 11/30
624/624 [==============================] - 148s 238ms/step - loss: 0.3309
Epoch 12/30
624/624 [==============================] - 151s 243

# Generate text

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [10]:
checkpoint_dir = './training_checkpoints'
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 1256)           106760    
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       (1, None, 1024)           7010304   
_________________________________________________________________
cu_dnngru_6 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
cu_dnngru_7 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
cu_dnngru_8 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
cu_dnngru_9 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 85)             87125     
Total para

In [63]:
def generate_text(model, start_string):
    
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])
    
  result = start_string + ''.join(text_generated)

  return (result)

In [66]:
print(generate_text(model, start_string=u"Alert"))

Alert:
The National Weather Service in Jackson has issued a Small
Craft Advisory, which is in effect from noon to 8 PM EDT Saturday.

* Plade sitage is 18.0 feet.
* Minor flooding is occurring and Minor flooding is forecast.
* Flood stage is 24.0 feet.
* Forecast...The river will continue rising to near 21.1 feet by Saturday evening.
* Impact, At  11.0 feet, Water is flooded roads. Most flood
deaths occur in vehicles.
Alert:
The National Weather Service in Jacksonville has issued a

* Severe Thunderstorm Warning for...
Northwestern Conroe County in east central Wisconsin...
Wanceton County in southeastern Texas...

* Until further notice.
* At  7:00 PM Friday the stage was 17.1 feet.
* Flood stage is 16.0 feet.
* Moderate flooding is occurring and is forecast to continue.
* Forecast...Rise above flood stage by Monday afternoon and
continue to rise to near 11.5 feet by Sunday April 21
then begin falling.
* At stages near 18.0 feet...Wide parks or forecasts change.

Safety message:  Neve